In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.3 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=d27ec1e5bf733426b9ddc2d2dfd3b5abe8d8e05d1db6121399f93f952a7def28
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.0 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK     dataframe_csv_UK
 a0529-a0611_US     dataframe_result_csv
 a0529.csv	    dataframe_result_csv_EU
 a0530.csv	    dataframe_result_csv_UK
 a0531.csv	    Github
 a0601.csv	    heatmap529_UK.ipynb
 a0602.csv	    heatmap530.ipynb
 a0603.csv	    Heat_Map_Online_language.ipynb
 a0604.csv	    __MACOSX
 a0605.csv	    non_violant1.csv
 a0606.csv	    non_violant2.csv
 csv		    UK
 csv.zip	    violant.csv
 dataframe_csv	   '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_csv_EU


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_531 = 'UK/06_08.csv'

In [8]:
ct = pd.read_csv(ct_path_531)

In [9]:
ct.shape[1]

47

In [10]:
#columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,104766277824536_277826290247506,Facebook,2020-06-08 23:59:50,2021-08-31 11:31:04,live_video_complete,HALESTORM's LZZY HALE Refuses To 'Shut Up And ...,blabbermouth.net,"Lzzy Hale has voiced her support for the ""Blac...",Doom Flamingo is FREE on Live From Out There t...,4.0,https://www.facebook.com/livefromoutthere/vide...,https://www.facebook.com/104766277824536/posts...,7756,1.333333,2.0,4,8,7,9,0,0,0,0,0,0,6,4,4,4,1,1,0,1,0,1,11553114,Live From Out There,livefromoutthere,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"11E1even Group, in conjunction with LiveXLive,...",False,2020-03-15 16:27:07,en,0,11553114|277826290247506
1,140363106133929_1459668160870077,Facebook,2020-06-08 23:59:47,2021-07-24 07:40:17,link,George Floyd protests: Chauvin's bail raised t...,abc.net.au,"White police officer Derek Chauvin, who is cha...",Bail for the former police officer charged wit...,2.0,https://www.abc.net.au/news/2020-06-09/us-geor...,https://www.facebook.com/140363106133929/posts...,54818,-3.363636,1.0,8,1,2,0,0,0,0,0,0,0,7,4,11,1,2,5,3,4,0,1,6076696,FIFO Construction Workers 3/1 Forum,fifocon3n1,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,AU,The consensus within the FIFO construction com...,False,2014-05-29 13:28:21,en,0,6076696|1459668160870077
2,686416788040835_3641993965816421,Facebook,2020-06-08 23:59:46,2021-06-18 03:32:21,link,Democrats take a knee in US Congress in George...,news.yahoo.com,House Speaker Nancy Pelosi and Senate Minority...,Washington (AFP) - Democratic lawmakers knelt ...,1.0,https://news.yahoo.com/democrats-knee-us-congr...,https://www.facebook.com/groups/68641678804083...,123489,7.322222,1.0,327,96,66,163,3,1,2,1,0,37,25,8,9,17,3,10,4,14,0,3,2508797,Activist ★ Democrats,fifocon3n1,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,AU,The consensus within the FIFO construction com...,False,2014-05-29 13:28:21,en,0,2508797|3641993965816421
3,71728150722_10156992007835723,Facebook,2020-06-08 23:59:40,2021-08-31 04:45:31,link,Philly Bride and Groom Get Married During a Ge...,insideedition.com,House Speaker Nancy Pelosi and Senate Minority...,"The couple stood with fists in the air, showin...",1.0,https://www.insideedition.com/philly-bride-and...,https://www.facebook.com/71728150722/posts/101...,2452509,1.286624,1.0,82,13,58,26,1,17,2,3,0,1,39,17,31,4,13,6,34,13,0,4,24723,Inside Edition,Inside.Edition,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Inside Edition is television's longest-running...,True,2009-04-10 15:37:04,en,0,24723|10156992007835723
4,572469979784527_1132964290401757,Facebook,2020-06-08 23:59:39,2021-07-27 16:53:33,link,We can’t post this on the MSM controlled socia...,cirnow.com.au,"Monday, June 08, 2020 by: Mike Adams We have n...",Growing evidence emerges that Obama is guilty ...,2.0,https://www.cirnow.com.au/we-cant-post-this-on...,https://www.facebook.com/572469979784527/posts...,5518,1.370370,1.0,12,14,7,0,1,0,0,3,0,0,7,6,3,2,2,2,3,2,0,1,8046512,CIRnow-australia,cirnowaustralia,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,AU,We the People are entitled through our Constit...,False,2018-03-08 21:41:47,en,0,8046512|1132964290401757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{},{'Minneapolis': 1}
1,{'US': 1},{'Minnesota': 1}
2,"{'Washington': 1, 'US': 1}",{'US': 1}
3,{},{'US': 1}
4,{'America': 1},{}


In [24]:
ct.to_csv('a0608.csv')

In [25]:
ct=pd.read_csv('a0608.csv',lineterminator="\n")

In [26]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [27]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [28]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},{'Minneapolis': 1},{}
1,{'US': 1},{'Minnesota': 1},{}
2,"{'Washington': 1, 'US': 1}",{'US': 1},{'US': 1}
3,{},{'US': 1},{}
4,{'America': 1},{},{}


In [29]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [30]:
ct_ner['NER-msg']

0                           []
1                         [US]
2             [US, Washington]
3                           []
4                    [America]
                 ...          
25298            [TX, Houston]
25299    [U.S., New York City]
25300                   [Ohio]
25301      [America, Brooklyn]
25302                       []
Name: NER-msg, Length: 17829, dtype: object

In [31]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
ct_ner['NER']

0                               [Minneapolis]
1                             [US, Minnesota]
2                    [US, Washington, US, US]
3                                        [US]
4                                   [America]
                         ...                 
25298              [TX, Houston, Minneapolis]
25299    [U.S., New York City, New York City]
25300                      [Ohio, Ohio, Ohio]
25301         [America, Brooklyn, Ohio, Ohio]
25302                            [Ohio, Ohio]
Name: NER, Length: 17829, dtype: object

In [33]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
ner_lst = ct_ner['NER'].values

In [36]:
ner_lst

array([list(['minneapolis']), list(['minnesota', 'us']),
       list(['washington', 'us']), ..., list(['ohio']),
       list(['ohio', 'america', 'brooklyn']), list(['ohio'])],
      dtype=object)

In [37]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [38]:
list

['minneapolis',
 'minnesota',
 'us',
 'washington',
 'us',
 'us',
 'america',
 'l.a.',
 'durham',
 'raleigh',
 'north carolina',
 'georgia',
 'the united states of america',
 'dallas',
 'seattle',
 'america',
 'renton',
 'us',
 'georgia',
 'seattle',
 'georgia',
 'america',
 'renton',
 'france',
 'the united states',
 'minnesota',
 'us',
 'minneapolis',
 'hennepin county',
 'minneapolis',
 'hennepin county',
 'minneapolis',
 'minneapolis',
 'mishawaka',
 'chattanooga',
 'minnesota',
 'the united states',
 'suffolk county',
 'minnesota',
 'the united states',
 'chattanooga',
 'chattanooga',
 'minnesota',
 'the united states',
 'minneapolis',
 'england',
 'manchester city’s',
 'britain',
 'minnesota',
 'us',
 'minnesota',
 'seattle',
 'america',
 'renton',
 'minneapolis',
 'minneapolis',
 'america',
 'simsbury',
 'la',
 'portland',
 'columbus',
 'asheville',
 'minneapolis',
 'tampa bay',
 'indianapolis',
 'tacoma',
 'uk',
 'louisville',
 'us',
 'piscataway',
 'nj',
 'piscataway',
 'birmi

In [39]:
c = collections.Counter()

In [40]:
dict_a=collections.Counter(list).most_common()

In [41]:
dict_a

[('minneapolis', 5928),
 ('america', 2402),
 ('houston', 2312),
 ('us', 2234),
 ('the united states', 1255),
 ('washington', 1136),
 ('u.s.', 1011),
 ('d.c.', 931),
 ('minnesota', 841),
 ('texas', 760),
 ('alabama', 752),
 ('canada', 658),
 ('australia', 602),
 ('france', 565),
 ('virginia', 557),
 ('buffalo', 556),
 ('boston', 554),
 ('germany', 549),
 ('maryland', 541),
 ('paris', 537),
 ('mexico', 509),
 ('berlin', 508),
 ('new zealand', 504),
 ('ireland', 498),
 ('madrid', 497),
 ('spain', 495),
 ('barcelona', 495),
 ('brazil', 492),
 ('athens', 487),
 ('lauderdale', 479),
 ('dublin', 478),
 ('denmark', 477),
 ('greece', 475),
 ('brussels', 475),
 ('amsterdam', 474),
 ('syria', 474),
 ('copenhagen', 473),
 ('jersey', 467),
 ('auckland', 463),
 ('mexico city', 461),
 ('california', 458),
 ('poland', 457),
 ('binnish', 439),
 ('seattle', 437),
 ('new york', 423),
 ('los angeles', 401),
 ('uk', 365),
 ('london', 324),
 ('usa', 308),
 ('new york city', 277),
 ('chicago', 218),
 ('portl

In [42]:
dataframe531=pd.DataFrame(dict_a)

In [43]:
len(dict_a)

3332

In [44]:
dataframe531.to_csv('dataframe0608_UK.csv')